# Tutorial 1: Poisson equation

We are going to use an advanced finite element code to observe how spectral
Galerkin and finite element methods work in practise. Due to the limited time we
have in this course, I have decided to use the software implementation in `Gridap`,
which I co-develop. This software is written in `Julia`. If you do not know about `Julia`,
no worries. You won't need much for following the tutorials.

The code we will use is Gridap, which we load as follows.

In [1]:
using Gridap

We will also use Plots for plotting results.

In [2]:
using Plots

Now, we start defining our problem and its approximation.
The corners of the box, in 1D, end-points, of the mesh define the domain.
We consider $\Omega \doteq [0.0,1.0]$.

In [3]:
domain = (0.0,1.0)

(0.0, 1.0)

Now, we the number of cells in our mesh. We define a 1D partition.

In [4]:
N = 8
partition = (N,)

(8,)

The next function creates a structured uniform mesh of n-cubes in dimension n.
In 1D, it is just the mesh of [0,1] with N segments.

In [5]:
model = CartesianDiscreteModel(domain,partition)

CartesianDiscreteModel()

The model includes geometrical information. We can extract a
triangulation (mesh) out of it.

In [6]:
trian = get_triangulation(model)

CartesianGrid()

Now, we can pick the order of the finite element space

In [7]:
order = 3

3

Here we choose the degree of the quadrature and compute it.

In [8]:
degree = (order-1)*2
quad = CellQuadrature(trian,degree)

CellQuadrature()

Let us consider a problem with analytical solution in strong sense.
$x$ is a point in the space. In 1D, do not forget to take its 1st component, `x[1]`.

In [9]:
u(x) = x[1]^2 - 2x[1] + 1

u (generic function with 1 method)

We want $u(x)$ to be solution of the Poisson problem. Thus, the forcing term must
be (for the Poisson equation):

In [10]:
f(x) = - Δ(u)(x)

f (generic function with 1 method)

Now we create a finite element space for all the information above.
`boundary` means that we consider Dirichlet data on the whole boundary, i.e.,
end-points 0 and 1 and `Float64` means that our unknowns is scalar.

In [11]:
V = TestFESpace(
   model=model,
   order=order,
   reffe=:Lagrangian,
   valuetype=Float64,
   dirichlet_tags="boundary")

UnconstrainedFESpace()

The trial space is an affine space with boundary conditions. The next
method evaluates u at the end-points and consider these values as Dirichlet values.

In [12]:
U = TrialFESpace(V,u)

TrialFESpace()

Now the (bi)linear form. For the Poisson equation we have reads:

In [13]:
a(u,v) = inner(∇(u),∇(v))
l(v) = inner(v,f)

l (generic function with 1 method)

Now, we link the (bi)linear forms, the triangulation in which they are
integrated, and the numerical quadrature rule for this integration

In [14]:
t_Ω = AffineFETerm(a,l,trian,quad)

AffineFETermFromIntegration()

With these terms, we can now create our operator.
E.g., F(x) = Ax-b, where A is our matrix and b the right-hand side,
thus affine. It takes the term and the trial/test finite element spaces.

In [15]:
op = AffineFEOperator(U,V,t_Ω)

AffineFEOperator()

Now, we solve the linear system and get the finite element solution.

In [16]:
uh = solve(op)

SingleFieldFEFunction()

Now, let us check the error. We define the h1 semi-norm (squared)

In [17]:
l2(w) = inner(w,w)
sh1(w) = a(w,w)
h1(w) = sh1(w) + l2(w)

h1 (generic function with 1 method)

The error function is

In [18]:
e = u - uh

GenericCellField()

Now we integrate the value of the square of the norm at each cell and
add together for all cells and take the squared root. We can compute the
$L^2(\Omega)$ norm.

In [19]:
el2 = sqrt(sum( integrate(l2(e),trian,quad) ))

4.4086086788806116e-16

We can also compute the $H^1(\Omega)$.

In [20]:
eh1 = sqrt(sum( integrate(h1(e),trian,quad) ))

5.251605546319498e-15

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*